In [ ]:
import pandas as pd
import numpy as np

from urllib.parse import quote    
from urllib.request import urlopen

import sys
import requests
from bs4 import BeautifulSoup 
import pandas as pd
from sqlalchemy import create_engine

#import pymysql.cursors
#pymysql.install_as_MySQLdb()
#import MySQLdb
from pandas.io.json import json_normalize
import time
#from pyproj import Proj, transform
#import folium
#from folium.plugins import MarkerCluster, MiniMap
import re

# Subway monthly boarding log

In [ ]:
api_key = '4c6e727a6c6779653532546b576f6e'
base_url = 'http://openapi.seoul.go.kr:8088/{}/json/CardSubwayTime/{}/{}/{}/'

year_range = ['2015','2016','2017','2018','2019']
month_range = ['01','02','03','04','05','06','07','08','09','10','11','12']

for j in year_range:
    start_time = time.time()
    table_name = 'subway_monthly_timelog_'+j
    monthly_df = pd.DataFrame()
    
    for i in month_range:
        ym_value = j + i
        
        num = 1
        while True:
            try:

                req = requests.get(base_url.format(api_key,str(num),str(num+999),ym_value), headers = {'user-agent': \
                                                ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36')})

                subway_json = req.json()
                sample_df = json_normalize(subway_json['CardSubwayTime']['row'])
                
                monthly_df = monthly_df.append(sample_df)

                #sample_df.to_sql(name=table_name,con=engine,if_exists='append')


                print('지하철 데이터 {} ~ {} row 수집중'.format(str(num),str(num+999)))
                num +=1000
            except:
                break
        print('--------------------'+j+'년'+i+'월 데이터 수집 완료---------------------')
        
        
    print('============================================================================')
    print('=============================='+j+' 연도 테이블 생성 완료========================')
    print('============================================================================')
    print('연평균 수집 소요 시간(초) : ', time.time()-start_time)
    monthly_df.index = range(len(monthly_df))

    monthly_df.to_csv(table_name+'.csv',index = False)



In [ ]:
subway_monthlylog2015 = pd.read_csv('subway_monthly_timelog_2015.csv')
subway_monthlylog2016 = pd.read_csv('subway_monthly_timelog_2016.csv')
subway_monthlylog2017 = pd.read_csv('subway_monthly_timelog_2017.csv')
subway_monthlylog2018 = pd.read_csv('subway_monthly_timelog_2018.csv')
subway_monthlylog2019 = pd.read_csv('subway_monthly_timelog_2019.csv')

In [ ]:
subway_monthly = pd.concat([subway_monthlylog2015,subway_monthlylog2016,subway_monthlylog2017,\
                           subway_monthlylog2018,subway_monthlylog2019])

# Subway daily Boarding log

In [ ]:
api_key = '7a4d4a55776779653838644e786969'
base_url = 'http://openapi.seoul.go.kr:8088/{}/json/CardSubwayStatsNew/{}/{}/{}'

year_range = ['2015','2016','2017','2018','2019']
date_range = pd.date_range(start='20150101', end='20191231').strftime('%Y%m%d').tolist()
month_range = []
for i in date_range:
    month_range.append(i[4:])
day_range = pd.Series(month_range).unique().tolist()[:-1]   

In [ ]:
for i in year_range:
    start_time = time.time()
    db_name = 'subway_dailylog_' + i
    
    dailylog_df = pd.DataFrame()
    for j in day_range:
        ym = i+j
        
        num = 1
        
        while True:
            try:
                req = requests.get(base_url.format(api_key,str(num),str(num+999),ym), headers = {'user-agent': \
                                                        ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36')})

                json_data = req.json()
                sample_df = json_normalize(json_data['CardSubwayStatsNew']['row'])
                
                dailylog_df = dailylog_df.append(sample_df)
                #sample_df.to_sql(name=db_name,con=engine,if_exists='append')
                
                #print('지하철 데이터 {} ~ {} row 수집중'.format(str(num),str(num+999)))
                print(i+j+' 데이터 적재 완료')
                num +=1000
            except:
                break
                
    print('============================================================================')
    print('=============================='+i+' 연도 테이블 생성 완료========================')
    print('============================================================================')
    print('연평균 수집 소요 시간(초) : ', time.time()-start_time)
    
    dailylog_df.index = range(len(dailylog_df))
    dailylog_df.to_csv(db_name+'.csv',index = False)

        

In [ ]:
subway_dailylog2015 = pd.read_csv('subway_dailylog_2015.csv')
subway_dailylog2016 = pd.read_csv('subway_dailylog_2016.csv')
subway_dailylog2017 = pd.read_csv('subway_dailylog_2017.csv')
subway_dailylog2018 = pd.read_csv('subway_dailylog_2018.csv')
subway_dailylog2019 = pd.read_csv('subway_dailylog_2019.csv')

In [ ]:
subway_dailylog = pd.concat([subway_dailylog2015,subway_dailylog2016,subway_dailylog2017,\
                            subway_dailylog2018,subway_dailylog2019])